In [19]:
import pickle
from hydrate_neo4j_graph import *


In [ ]:
with open('confs_by_year.pickle', 'rb') as f:
    confs_by_year = pickle.load(f)
with open('papers.pickle', 'rb') as f:
    papers_by_conf = pickle.load(f)
num_papers = 0
papers = {}
for conf in papers_by_conf:
    for paper_id in papers_by_conf[conf]:
        num_papers+=1
        papers[paper_id]=papers_by_conf[conf][paper_id]
assert num_papers==len(papers)
authors = defaultdict(list)
for paper in papers:
    for author_id, author in zip(papers[paper].content["authorids"], papers[paper].content["authors"]):
        authors[f'{author_id}|{author}'].append(papers[paper].id)

insert_authors_params = make_author_nodes(authors)
insert_confs_params = make_conf_nodes(confs_by_year)
insert_paper_params, insert_reviews_params, insert_relations = \
    make_paper_review_nodes_and_relations(confs_by_year, papers)

In [ ]:
# driver = GraphDatabase.driver(uri, auth=("neo4j", "openreview"))
try:
    rels = make_reviewer_relation(driver, [r["obj"] for r in insert_relations["relations"]["_HAS_REVIEW"]])
finally:
    driver.close()

In [ ]:
assert len(list(rels))==len(insert_relations["relations"]["_HAS_REVIEW"])

In [ ]:
len(insert_relations["relations"]["_HAS_REVIEW"])

In [ ]:
len(list(rels))

In [ ]:
rels